In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Cel: Przygotowanie danych dotyczących migracji
* Przygotowanie datasetu używanego do analiz korelacji pewnych współczynników z liczbą migrantów:
    * Porównanie informacji POPULATION BY CITIZENSHIP, ALL VALID PERMITS, MIGRATION BY CITIZENSHIP
    * Dodanie do wyjściowego datasetu informacji na temat Ukraińskich uchodźcach
    * Dodanie informacji dotyczących pobytów tymczasowych

# POPULATION BY CITIZENSHIP

selected_countries = ["Poland", "Bulgaria", "Romania", "Hungary", "France", "Italy", "Germany", "Sweden", "Spain", "Greece", "Slovenia"]


Skondensowana tabela przedstawia dane dotyczące ilości populacji w wybranych krajach w następującym formacie:
NAZWA KRAJU ROK LICZBA_OBYWATELI_UE LICZBA_OBYWATELI_NON_UE ŁĄCZNA LICZBA OBCOKRAJOCÓW

In [55]:
pop_cit = pd.read_csv('../processed_data/population_by_citizenship.csv')

In [56]:
pop_cit.head()

,citizen,geo,year,number of citizens in country
0,Andorra,Austria,2015,4
1,Andorra,Austria,2016,3
2,Andorra,Austria,2017,4
3,Andorra,Austria,2018,5
4,Andorra,Austria,2019,4


In [57]:
selected_countries = ["Poland", "Bulgaria", "Romania", "Hungary", "France", "Italy", "Germany", "Sweden", "Spain", "Greece", "Slovenia"]
pop_cit = pop_cit[pop_cit['geo'].isin(selected_countries)]

In [58]:
pop_cit.head()

,citizen,geo,year,number of citizens in country
20,Andorra,Bulgaria,2023,0
21,Andorra,Bulgaria,2024,0
39,Andorra,Germany,2015,30
40,Andorra,Germany,2016,38
41,Andorra,Germany,2017,40


In [59]:
eu_countries = [
    "Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czechia",
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece",
    "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg",
    "Malta", "Netherlands", "Poland", "Portugal", "Romania", "Slovakia",
    "Slovenia", "Spain", "Sweden"
]

In [60]:
pop_cit = pop_cit[
    ~pop_cit["geo"].str.contains("European Union|Total", case=False, na=False) &
    ~pop_cit["citizen"].str.contains("Asia|America|Africa|Europe|Oceanian|Candidate|European Union|Unknown|EU|European|Total", case=False, na=False)
]

In [61]:
def classify_citizen(row):
    if row['citizen'] == 'Reporting country':
        return 'national'
    elif row['citizen'] in eu_countries:
        return 'EU'
    else:
        return 'non_EU'
    
pop_cit['citizen_type'] = pop_cit.apply(classify_citizen, axis=1)

In [62]:
summary = pop_cit.groupby(['geo', 'year', 'citizen_type'])['number of citizens in country'].sum().reset_index()

In [65]:
pivot = summary.pivot_table(
    index=['geo', 'year'],
    columns='citizen_type',
    values='number of citizens in country',
    fill_value=0
).reset_index()

In [ ]:
pivot[pivot['geo'] == 'Poland']  # checking the pivot table for Poland

citizen_type,geo,year,EU,national,non_EU
60,Poland,2015,0.0,37891051.0,110001.0
61,Poland,2016,37834800.0,37811676.0,152122.0
62,Poland,2017,37783928.0,37756724.0,213118.0
63,Poland,2018,37759287.0,37731606.0,242205.0
64,Poland,2019,37712047.0,37683061.0,293009.0
65,Poland,2020,37633096.0,37599899.0,362239.0
66,Poland,2021,0.0,36623052.0,448078.0
67,Poland,2022,0.0,36445251.0,442296.0
68,Poland,2023,0.0,36314890.0,436595.0
69,Poland,2024,0.0,36187908.0,430864.0


In [68]:
pivot['foreigners'] = pivot['non_EU'] + pivot['EU']

In [69]:
pivot.head()

citizen_type,geo,year,EU,national,non_EU,foreigners
0,Bulgaria,2015,0.0,6963308.0,63879.0,63879.0
1,Bulgaria,2016,0.0,6863112.0,71491.0,71491.0
2,Bulgaria,2017,0.0,6767835.0,76562.0,76562.0
3,Bulgaria,2018,0.0,6668711.0,82370.0,82370.0
4,Bulgaria,2019,0.0,6565004.0,91243.0,91243.0


In [71]:
pivot.to_csv('../processed_data/population_by_citizenship_combined_data.csv', index=False)

# MIGRRANTS BY CITIZENSHIP